# Part 3(b)

This notebook will highlight the process, result and insights obtained from using the embedding matrix built that includes OOV words with their contextual average as embeddings.

## Import Embedding Matrix

In [2]:
import json
from pathlib import Path

import numpy as np

embedding_path = Path("models/embedding_matrix_oov.npy")
index_from_word_path = Path("models/index_from_word_oov.json")

embedding_matrix = np.load(embedding_path)
with index_from_word_path.open() as f:
    index_from_word = json.load(f)

## Prepare Dataaset

(a) Import datasets

In [ ]:
from utils.text import tokenize
from datasets import load_dataset

dataset = load_dataset("rotten_tomatoes")
train_dataset = tokenize(dataset["train"])
val_dataset = tokenize(dataset["validation"])
test_dataset = tokenize(dataset["test"])

In [4]:
from utils.text import token_to_index

train_dataset = token_to_index(dataset=train_dataset, index_from_word=index_from_word)
val_dataset = token_to_index(dataset=val_dataset, index_from_word=index_from_word)
test_dataset = token_to_index(dataset=test_dataset, index_from_word=index_from_word)

train_dataset

Dataset({
    features: ['text', 'label', 'tokens', 'original_len', 'indexes'],
    num_rows: 8530
})

In [5]:
train_dataset = train_dataset.select_columns(["label", "original_len", "indexes"])
val_dataset = val_dataset.select_columns(["label", "original_len", "indexes"])
test_dataset = test_dataset.select_columns(["label", "original_len", "indexes"])

In [6]:
train_dataset.set_format(type="torch")
val_dataset.set_format(type="torch")
test_dataset.set_format(type="torch")

## Train RNN

We use Optuna to perform heuristic search on optimal configuration when the embedding matrix with OOV words is updatable during training.

In [ ]:
import optuna
from utils.train import train_rnn_model_with_parameters

_N_TRIALS = 1500
SEARCH_SPACE = {
    "batch_size": [32, 64, 128, 256, 512, 1024, 2048],
    "learning_rate": [1e-5, 1e-4, 1e-3, 1e-2, 1e-1],
    "optimizer_name": ["SGD", "Adagrad", "RMSprop", "Adam"],
    # RNN Model Parameters
    "hidden_dim": [256, 128, 64, 32],
    "num_layers": [1, 2, 4],
    "sentence_representation_type": ["max"],
}

def objective(trial):
    hidden_dim = trial.suggest_categorical("hidden_dim", SEARCH_SPACE["hidden_dim"])
    num_layers = trial.suggest_int("num_layers", min(SEARCH_SPACE["num_layers"]), max(SEARCH_SPACE["num_layers"]))
    optimizer_name = trial.suggest_categorical("optimizer_name", SEARCH_SPACE["optimizer_name"])
    batch_size = trial.suggest_categorical("batch_size", SEARCH_SPACE["batch_size"])
    learning_rate = trial.suggest_categorical("learning_rate", SEARCH_SPACE["learning_rate"])
    sentence_representation_type = trial.suggest_categorical("sentence_representation_type", SEARCH_SPACE["sentence_representation_type"])

    log_message = f"---------- batch_size_{batch_size}; lr_{learning_rate}; optimizer_{optimizer_name}; hidden_dim_{hidden_dim}; num_layers_{num_layers}; sentence_representation_{sentence_representation_type} ----------"
    print(log_message)

    val_acc = train_rnn_model_with_parameters(
        embedding_matrix=embedding_matrix,
        train_dataset=train_dataset,
        val_dataset=val_dataset,
        batch_size=batch_size,
        learning_rate=learning_rate,
        optimizer_name=optimizer_name,
        hidden_dim=hidden_dim,
        num_layers=num_layers,
        sentence_representation_type=sentence_representation_type,
        show_progress=True,
        freeze_embedding=False,
        log_dir="rnn_trainable_embeddings_and_contextual_oov",
    )

    return val_acc

# Set up the Optuna study
study = optuna.create_study(direction="maximize") 
study.optimize(objective, n_trials=_N_TRIALS)

best_params = study.best_params

## Train Result Analysis

(a) Load result from all trials

In [10]:
from utils.analytics import load_tensorboard_logs

train_results_df = load_tensorboard_logs(
    log_dir="tb_logs/rnn_trainable_embeddings_and_contextual_oov"
)

train_results_df = train_results_df.sort_values(
    by=["val_acc"], ascending=False
).reset_index(drop=True)
train_results_df.head(20)

,val_acc,batch_size,hidden_dim,learning_rate,optimizer_name,train_loss,train_acc,num_layers,sentence_representation_type,freeze,epoch,val_loss,filename
0,0.828330,32,64,0.0100,Adagrad,0.019335,1.000000,2,max,False,3.0,0.464988,events.out.tfevents.1730885820.yuriarch.387836...
1,0.827392,32,256,0.0100,Adagrad,0.052982,0.972222,1,max,False,3.0,0.453625,events.out.tfevents.1730888923.yuriarch.387836...
2,0.827392,64,32,0.0100,Adagrad,0.137741,0.968750,2,max,False,3.0,0.452066,events.out.tfevents.1730886082.yuriarch.387836...
3,0.827392,32,32,0.0100,Adagrad,0.208951,0.972222,2,max,False,3.0,0.448410,events.out.tfevents.1730885789.yuriarch.387836...
4,0.825516,32,128,0.0010,RMSprop,0.041138,0.973684,2,max,False,3.0,0.449159,events.out.tfevents.1730883349.yuriarch.387836...
5,0.823640,32,128,0.0010,RMSprop,0.111653,0.947368,1,max,False,3.0,0.451423,events.out.tfevents.1730885078.yuriarch.387836...
6,0.819887,32,64,0.0010,Adam,0.016075,1.000000,2,max,False,3.0,0.455369,events.out.tfevents.1730881312.yuriarch.387836...
7,0.818949,32,32,0.0100,Adagrad,0.030982,1.000000,3,max,False,4.0,0.479416,events.out.tfevents.1730885978.yuriarch.387836...
8,0.817073,32,64,0.0010,Adam,0.035536,1.000000,4,max,False,4.0,0.462661,events.out.tfevents.1730881634.yuriarch.387836...
9,0.817073,32,128,0.0100,Adagrad,0.025396,1.000000,2,max,False,3.0,0.446082,events.out.tfevents.1730885565.yuriarch.387836...


(b) Configuration for best trial result

In [7]:
best_rnn_model_configuration = train_results_df.head(1)
best_rnn_model_configuration

,val_acc,batch_size,hidden_dim,learning_rate,optimizer_name,train_loss,train_acc,num_layers,sentence_representation_type,freeze,epoch,val_loss,filename
0,0.82833,32,64,0.01,Adagrad,0.019335,1.0,2,max,False,3.0,0.464988,events.out.tfevents.1730885820.yuriarch.387836...


## Performance on Test Dataset

In [ ]:
from utils.analytics import test_top_n_models
from models.RNN import RNNClassifier

test_results_df = test_top_n_models(train_results_df, RNNClassifier, test_dataset, n=10)

In [9]:
test_results_df

,test_acc,test_loss,val_acc,batch_size,hidden_dim,learning_rate,optimizer_name,train_loss,train_acc,num_layers,sentence_representation_type,freeze,epoch,val_loss,filename
0,0.803002,0.41246,0.828330,32,64,0.010,Adagrad,0.019335,1.000000,2,max,False,3.0,0.464988,events.out.tfevents.1730885820.yuriarch.387836...
1,0.792683,0.425412,0.827392,32,256,0.010,Adagrad,0.052982,0.972222,1,max,False,3.0,0.453625,events.out.tfevents.1730888923.yuriarch.387836...
2,0.793621,0.417104,0.827392,64,32,0.010,Adagrad,0.137741,0.968750,2,max,False,3.0,0.452066,events.out.tfevents.1730886082.yuriarch.387836...
3,0.798311,0.419237,0.827392,32,32,0.010,Adagrad,0.208951,0.972222,2,max,False,3.0,0.448410,events.out.tfevents.1730885789.yuriarch.387836...
4,0.805816,0.404464,0.825516,32,128,0.001,RMSprop,0.041138,0.973684,2,max,False,3.0,0.449159,events.out.tfevents.1730883349.yuriarch.387836...
5,0.809568,0.403682,0.823640,32,128,0.001,RMSprop,0.111653,0.947368,1,max,False,3.0,0.451423,events.out.tfevents.1730885078.yuriarch.387836...
6,0.80394,0.416273,0.819887,32,64,0.001,Adam,0.016075,1.000000,2,max,False,3.0,0.455369,events.out.tfevents.1730881312.yuriarch.387836...
7,0.790807,0.433963,0.818949,32,32,0.010,Adagrad,0.030982,1.000000,3,max,False,4.0,0.479416,events.out.tfevents.1730885978.yuriarch.387836...
8,0.794559,0.425954,0.817073,32,64,0.001,Adam,0.035536,1.000000,4,max,False,4.0,0.462661,events.out.tfevents.1730881634.yuriarch.387836...
9,None,None,0.817073,32,128,0.010,Adagrad,0.025396,1.000000,2,max,False,3.0,0.446082,events.out.tfevents.1730885565.yuriarch.387836...
